In [ ]:
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbs


In [ ]:
train = pd.read_csv("../data/raw/train.csv")

In [2]:
train_labels = pd.read_csv("../data/raw/train_labels.csv")

In [ ]:
test = pd.read_csv("../data/raw/test.csv")

In [5]:
train_labels.head()


,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [ ]:
train_labels.groupby('accuracy_group')['game_session'].count().plot(kind='barh')
plt.show()